<a href="https://colab.research.google.com/github/Kidara/INMCA_2021/blob/main/EDA_features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is adapted from the original *EDA_features_ex.ipynb* notebook from the PMEmo github repository.

# Upload data from GDrive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip -q '/content/gdrive/MyDrive/Colab Notebooks/INMCA_2021/PMEmo2019.zip'

In [3]:
dir_path = '/content/PMEmo2019/EDA'

# Extract EDA Features for Static/Dynamic MER

In [4]:
!pip install -q neurokit

     |████████████████████████████████| 720 kB 5.0 MB/s 
     |████████████████████████████████| 6.9 MB 18.1 MB/s 
     |████████████████████████████████| 85 kB 3.4 MB/s 


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import neurokit as nk
from scipy import stats
from tqdm import tqdm_notebook
from sklearn.preprocessing import minmax_scale, scale
from scipy.stats import kurtosis, skew
from librosa.feature import mfcc

In [6]:
def window_with_overlap(a, window, stride):
    nrows = ((a.size-window)//stride)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,window), strides=(stride*n,n))

def scr_features(times, phasic, SCR_Onsets, SCR_Peaks_Indexes, SCR_Peaks_Amplitudes):
    features = {}
    features['peakCount'] = len(SCR_Peaks_Indexes)
    features['meanpeakAmplitude'] = np.mean(SCR_Peaks_Amplitudes)
    RiseTime =  times[SCR_Peaks_Indexes] - times[SCR_Onsets]
    features['MeanRiseTime'] = np.mean(RiseTime)
    features['SumPeakAmplitude'] = np.sum(SCR_Peaks_Amplitudes)
    features['SumRiseTime'] = np.sum(RiseTime)
    features['SumAreas'] = np.trapz(phasic, x=times)
    return features

def time_domain_stats(times, signal):
    features = {}
    features['auc'] = np.trapz(signal, x=times)
    features['meanEDA'] = np.mean(signal)
    features['stdEDA'] = np.std(signal)
    features['kurtEDA'] = kurtosis(signal)
    features['skewEDA'] = skew(signal)
    features['meanDerivative'] = np.mean(np.gradient(signal))
    features['meanNegativeDerivative'] = np.mean((0-np.gradient(signal)))
    return features

def frequency_domain_stats(frequency_signal, frequency):
    features = {}
    features['SMA_f'] = np.trapz(np.abs(frequency_signal), x=frequency)
    features['meanEDA_f'] = np.mean(frequency_signal)
    features['stdEDA_f'] = np.std(frequency_signal)
    features['signalRange_f'] = frequency_signal.max() - frequency_signal.min()
    features['kurtEDA_f'] = kurtosis(frequency_signal)
    features['skewEDA_f'] = skew(frequency_signal)
#     FrequencyDomain_Statistical['harmonicsSummation']
    return features

def band_power(frequency_signal, frequency):
    features = {}
    signal_inbands = frequency_signal[(frequency>=0) & (frequency<=0.5)]
    features['signalEnergy'] = np.sum(np.abs(signal_inbands)**2)
    band1 = frequency_signal[(frequency>=0) & (frequency<0.1)]
    band2 = frequency_signal[(frequency>=0.1) & (frequency<0.2)]
    band3 = frequency_signal[(frequency>=0.2) & (frequency<0.3)]
    band4 = frequency_signal[(frequency>=0.3) & (frequency<0.4)]
    band5 = frequency_signal[(frequency>=0.4) & (frequency<=0.5)]
    bands = [band1, band2, band3, band4, band5]
    bands_sp = np.zeros(len(bands))
    for i, b in enumerate(bands):
        sp = np.sum(b**2)
        bands_sp[i] = sp
        features['SpectralPower_band'+str(i+1)] = sp
    features['minSpectralPower'] = np.min(bands_sp)
    features['maxSpectralPower'] = np.max(bands_sp)
    features['varSpectralPower'] = np.var(bands_sp)
    
    return features

        
def extract_eda_features(raw_eda, sampleRate=50, ifDynamic=False, window=50, stride=25):
    normalized_data = minmax_scale(raw_eda.reshape(-1, 1)).reshape(-1,)
    processed_results = nk.eda_process(normalized_data, sampling_rate=sampleRate)

    filtered_eda = processed_results['df']['EDA_Filtered']
    tonic = processed_results['df']['EDA_Tonic']
    phasic = processed_results['df']['EDA_Phasic']
    
    SCR_Onsets = processed_results['EDA']['SCR_Onsets']
    SCR_Peaks_Indexes  = processed_results['EDA']['SCR_Peaks_Indexes']
    SCR_Recovery_Indexes = processed_results['EDA']['SCR_Recovery_Indexes']
    SCR_Peaks_Amplitudes = processed_results['EDA']['SCR_Peaks_Amplitudes']
    
    signal = phasic
    times = np.arange(len(signal))/sampleRate

    features = pd.DataFrame()
    
    # static features 33 dimension
    if not ifDynamic:
        eda_features = {}
        # Time Domain: SCR Features, Statistical Features
        SCR_Features = scr_features(times, phasic, SCR_Onsets, 
                                    SCR_Peaks_Indexes, SCR_Peaks_Amplitudes)
        
        TimeStats = time_domain_stats(times, signal)
        # Frequency Domain: Statistical Features, Band Power
        frequency_signal = np.abs(np.fft.fft(signal))
        frequency = np.fft.fftfreq(signal.size, d=1/sampleRate)
        FrequencyStats = frequency_domain_stats(frequency_signal, frequency)
        BandPower = band_power(frequency_signal, frequency)
        
        eda_features.update(SCR_Features)
        eda_features.update(TimeStats)
        eda_features.update(FrequencyStats)
        eda_features.update(BandPower)
        
        # Time-Frequency Domain: MFCC Features
        n_mfcc = 20
        mfccCoefficients = mfcc(signal.values, sr=sampleRate, n_mfcc=n_mfcc)
        for i in range(n_mfcc):
            mfccCoef = mfccCoefficients[i,:]
            eda_features[f'meanMFCC[{i}]'] = np.mean(mfccCoef)
            eda_features[f'stdMFCC[{i}]'] = np.std(mfccCoef)
            eda_features[f'medianMFCC[{i}]'] = np.median(mfccCoef)
            eda_features[f'kurtMFCC[{i}]'] = kurtosis(mfccCoef.reshape(-1,))
            eda_features[f'skewMFCC[{i}]'] = skew(mfccCoef.reshape(-1,))
        features = pd.DataFrame(data=eda_features, index=[0])
        
    else: # dynamic features
        frames = window_with_overlap(signal, window, stride)
        time_frames = window_with_overlap(times, window, stride)
        
        for fidx, frame in enumerate(frames):
            eda_features = {}
            frame_time = fidx*0.5+1
            times = time_frames[fidx]
            eda_features['frameTime'] = frame_time
        
            # Time Domain: Statistical Features
            TimeStats = time_domain_stats(times, frame)
            
            # Frequency Domain: Statistical Features, Band Power
            frequency_signal = np.abs(np.fft.fft(frame))
            frequency = np.fft.fftfreq(frame.size, d=1/sampleRate)
            FrequencyStats = frequency_domain_stats(frequency_signal, frequency)
            BandPower = band_power(frequency_signal, frequency)

            eda_features.update(TimeStats)
            eda_features.update(FrequencyStats)
            eda_features.update(BandPower)   

            # Time-Frequency Domain: MFCC Features
            n_mfcc = 20
            mfccCoefficients = mfcc(frame, sr=sampleRate, n_mfcc=n_mfcc).reshape(-1,)
            for i in range(mfccCoefficients.size):
                eda_features[f'mfccCoefficient[{i}]'] = mfccCoefficients[i]
            eda_features['meanMFCC'] = np.mean(mfccCoefficients)
            eda_features['stdMFCC'] = np.std(mfccCoefficients)
            eda_features['medianMFCC'] = np.median(mfccCoefficients)
            eda_features['kurtMFCC'] = kurtosis(mfccCoefficients.reshape(-1,))
            eda_features['skewMFCC'] = skew(mfccCoefficients.reshape(-1,))
            features = features.append(pd.DataFrame(data=eda_features, index=[i]))
        
    return features


## Compute the static EDA features for songs

In [7]:
dir_path = dir_path
eda_files = [f for f in os.listdir(dir_path) if f[-4:]=='.csv']
results = pd.DataFrame()
# window = 50
# stride = 25
for f in tqdm_notebook(eda_files):
# for f in ['1_EDA.csv', '4_EDA.csv']:
    musicId = f.split('_')[0]
    raw_eda = pd.read_csv(os.path.join(dir_path, f), index_col=0)
#     row_num, col_num = raw_eda.shape[0], raw_eda.shape[1]-1

    for c in raw_eda.columns:
        subjectId = c[-6:]
        features = extract_eda_features(raw_eda[c].values)
        features['subjectId'] = subjectId
        features['musicId'] = musicId
        results = results.append(features)

results.fillna(0)
results.to_csv('EDA_features_static.csv', index=False, columns=['subjectId', 'musicId']+list(results.columns[:-2]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/794 [00:00<?, ?it/s]

Output streaming troncato alle ultime 5000 righe.
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=2010
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=2010
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=2010
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=2010
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1660
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1660
  n_fft, y.shape[

In [8]:
eda_feature_set = pd.read_csv('EDA_features_static.csv')
static_labels = pd.read_csv('/content/PMEmo2019/annotations/static_annotations.csv')
eda_dataset = pd.merge(eda_feature_set, static_labels, on=['musicId']).dropna()
eda_dataset.to_csv('eda_dataset.csv', index=False)

In [9]:
!cp -r EDA_features_static.csv '/content/gdrive/MyDrive/Colab Notebooks/INMCA_2021/CSV/EDA_features_static.csv'
!cp -r eda_dataset.csv '/content/gdrive/MyDrive/Colab Notebooks/INMCA_2021/CSV/eda_dataset.csv'